In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from util.dataloader import DataLoader
from util.datasplitter import data_splitter
from preprocessing import Preprocessor
from evaluator import evaluate_classifier
from fasttext_embeddings import FastTextEmbeddings
from nltk import download
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

#Load linguistic resources 
download('stopwords',quiet=True)
download('omw-1.4',quiet=True)
download('punkt',quiet=True)
download('wordnet',quiet=True);

In [2]:
#Load data : Here all emotion datasets
dl = DataLoader()
data = dl.load()

data is a nested dictionary where keys are referring to the datasets and subkeys (if any) are referring to the available splits

In [4]:
#Available datasets
print(data.keys())
#Available splits for the CARER dataset
print(data['CARER'].keys())

dict_keys(['liar', 'twentynews', 'agnews', 'yahoo', 'eval_emotion', 'CARER', 'silicone', 'imdb', 'yelp', 'sst2', 'semeval_A', 'semeval_B', 'sarc', 'iSarcasm'])
dict_keys(['train', 'val', 'test'])


In [ ]:
#Show first rows of the eval_emotion train set
eval_emotion = data['eval_emotion']
eval_emotion['train'].head()

In [ ]:
#Initialize preprocessors
preprocessor = Preprocessor() #Preprocessor for standard text
tweet_preprocessor = Preprocessor(is_tweet=True) #Preprocessor for tweets

In [ ]:
#Split the data into train, val, and test sets
train_emo, val_emo, test_emo = data_splitter(data['eval_emotion'],
                                             tweet_preprocessor,  #Eval Emotion is a tweet dataset
                                             create_val_set=True)

In [ ]:
%%time
#Load fasttext 
fasttext = FastTextEmbeddings()
fasttext.load_model('fasttext/cc.en.300.bin')

In [ ]:
embedded_train_emo = fasttext.generate_sentence_embeddings(train_emo['text'])
embedded_test_emo = fasttext.generate_sentence_embeddings(test_emo['text'])
embedded_train_emo['label'] = train_emo['label']
embedded_test_emo['label'] = test_emo['label']

In [ ]:
#Evaluate a logistic regression without hyperparameters tuning trained on the dataset
metrics_emo_tfidf = evaluate_classifier(LogisticRegression(),train_emo,test_emo,
                                  save_model=True,model_path='models/lr_tfidf_emo')
metrics_emo_ft = evaluate_classifier(LogisticRegression(),embedded_train_emo,embedded_test_emo, 
                                     tfidf=False, #Set tf-idf to false if working with fasttext embeddings
                                     save_model=True,model_path='models/lr_ft_emo')